In [1]:
import pandas as pd
import os
import json
from pymongo import MongoClient
DATA = '../../../../data/'


In [2]:
def decorateRow(row):
    #['HEX_600', 'HEX_weighted_average_distance', 'HEX_weighted_mean_home_distance', 'HEX_weighted_max_home_distance', 'HEX_weighted_median_home_distance', 'HEX_weighted_radius_of_gyration', 'HEX_weighted_num_reporting_2015', 'HEX_weighted_HEX_coverage', 'HEX_weighted_subject_coverage', 'HEX_weighted_subject_hhi', 'HEX_total_reporting', 'HEX_total_user']
    record = row[1:].to_frame().transpose()
    return record

In [3]:
#Connect to client
client = MongoClient('localhost', 27017)
db = client['hexData']
hexagon_coll = db['hexagon']
# Drop it if it exsits.
hexagon_coll.drop() 

In [4]:
# all subjects
#Initialize collection
data_all_subjects = pd.read_csv(DATA + "hex_variables_all/poverty_added_01_hex_variables_non_gov_regardless_of_reporting_frequency_20210209.csv", sep='\t', header=0, index_col = 0)
#Adding object for each hexagon
#Object has 2 values: hexagon number and array of objects for data from each user type and frequency on hexagon
for index, row in data_all_subjects.iterrows():
    hexagon_coll.insert_one({
        'HEX_600': row.HEX_600,
        'results': []
    })
#Populating with data
all_subject_dir = DATA + "hex_variables_all/"
for filename in os.listdir(all_subject_dir):
    if filename.endswith(".csv"):
        data_all_subjects = pd.read_csv(os.path.join(all_subject_dir, filename), sep='\t', header=0, index_col=0)
        data_all_subjects['subject'] = "all"
        for index, row in data_all_subjects.iterrows():
            # Append new object from file to existing hexagon's array
            hexagon_coll.update_one({
                'HEX_600': row.HEX_600,
            },
            {
                '$push': 
                {
                    'results': json.loads(decorateRow(row).to_json(orient='records'))[0]
                }
            })

In [5]:
# by subjects
subject_dir = DATA + "hex_variables_by_subjects_for_viz_system_20220623/"
for subject in os.listdir(subject_dir):
    if subject!=".DS_Store":
        directory = subject_dir + subject + "/"
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                frequency = "all"
                if filename.startswith("01"):
                    user_type = "non_gov"
                elif filename.startswith("02"):
                    user_type = "non_gov_unsure"
                elif filename.startswith("03"):
                    user_type = "all"
                elif filename.startswith("04"):
                    user_type = "non_gov"
                    frequency = "heavy"
                elif filename.startswith("05"):
                    user_type = "non_gov_unsure"
                    frequency = "heavy"
                elif filename.startswith("06"):
                    user_type = "all"
                    frequency = "heavy"
                
                data_per_subject = pd.read_csv(os.path.join(directory, filename), header=0, index_col = 0)
                data_per_subject.rename(columns={"HEX":"HEX_600","HEX_total_number_of_reports":"HEX_total_reporting"}, inplace=True)

                data_per_subject['subject'] = subject
                data_per_subject['user_type'] = user_type
                data_per_subject['frequency'] = frequency
                data = data_per_subject.merge(data_all_subjects, on = 'HEX_600')
                data = data.loc[:, ~data.columns.str.contains('_y$')]
                data.columns = data.columns.str.replace(r'_x$', '')
                for index, row in data.iterrows():
                    # Append new object from file to existing hexagon's array
                    hexagon_coll.update_one({
                        'HEX_600': row.HEX_600,
                    },
                    {
                        '$push': 
                        {
                            'results': json.loads(decorateRow(row).to_json(orient='records'))[0]
                        }
                    })
                

<ipython-input-5-73716f42e040>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')


In [ ]:
# mongodump --db=hexData